In [59]:
import numpy as np
import pandas as pd
from geopy.distance import vincenty

In [60]:
data = pd.read_csv('data/data.csv')

In [61]:
def calc_distance(x):
    orchard = (1.303991, 103.831782)
    place2_coords = (x.lat, x.long)
    return vincenty(orchard, place2_coords).km

In [62]:
feature = data.iloc[:,:-1].copy()
price = data.iloc[:,-1].copy()

In [63]:
price = pd.DataFrame(price, columns=['price_sqft'])

In [64]:
feature['bathroom_area'] = 0.05 * feature['bathroom'] * feature['area']
feature['bedroom_area'] = 0.125 * feature['bedroom'] * feature['area']
feature['remaining_area'] = feature['area'] - feature['bathroom_area'] - feature['bedroom_area']
feature['convenience'] = feature['mrt'] + feature['school']
feature['distance_from_central'] = feature.apply(calc_distance, axis = 1)
feature.head()

C:\Users\TNG\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  after removing the cwd from sys.path.


,area,bathroom,bedroom,lat,long,school,mrt,age,type_condo,type_hdb,bathroom_area,bedroom_area,remaining_area,convenience,distance_from_central
0,1270,2,3,1.345383,103.746046,3,0,25,0,1,127.0,476.250,666.750,3,10.582454
1,1270,2,3,1.373571,103.851554,2,1,40,0,1,127.0,476.250,666.750,3,8.002258
2,990,2,2,1.315350,103.770855,2,0,43,0,1,99.0,247.500,643.500,2,6.895952
3,1400,2,3,1.448271,103.813344,4,0,21,0,1,140.0,525.000,735.000,4,16.085163
4,905,2,3,1.372392,103.889038,6,1,35,0,1,90.5,339.375,475.125,7,9.889811


In [65]:
data = feature.join(price)
data = data[[c for c in data if c not in ['type_condo', 'type_hdb', 'price_sqft']] \
            + ['type_condo', 'type_hdb', 'price_sqft']]

In [66]:
data.to_csv('data/feature_eng_data.csv')